# Run Pixel and Patch Classifier on a Test Site
Option to run a single pixel classifier or an ensemble of pixel classifiers

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tqdm.notebook import tqdm

from scripts import dl_utils

In [ ]:
def visualize_time_series(pairs, preds, dates, threshold=0.6, title=None, path=None):
    num_img = int(np.ceil(np.sqrt(len(pairs)))) + 1
    patches = [np.mean(pair, axis=0) for pair in pairs]
    plt.figure(figsize=(num_img,num_img), dpi=250, facecolor=(1,1,1))
    for i, (img, pred, date) in enumerate(zip(patches, preds, dates)):
        rgb = img[:,:,3:0:-1] / 3000
        rgb[pred > threshold, 0] = 0.9
        rgb[pred > threshold, 1] = 0
        rgb[pred > threshold, 2] = 0.1
        plt.subplot(num_img, num_img, i + 1)
        plt.title(date, size=5, y=0.9)
        plt.imshow(np.clip(rgb, 0, 1))
        plt.axis('off')
    
    # Only aggregate predictions from sufficiently cloud-free patches
    cloud_free_patches = []
    for pair in pairs:
        img_1_cloud = np.sum(pair[0].mask) / np.size(pair[0].mask)
        img_2_cloud = np.sum(pair[1].mask) / np.size(pair[1].mask)
        cloud_threshold = 0.25
        if img_1_cloud < cloud_threshold and img_2_cloud < cloud_threshold:
            cloud_free_patches.append(True)
        else:
            cloud_free_patches.append(False)
    mean_patch = np.ma.mean([pair[0] for pair in pairs], axis=0)
    mean_patch = mean_patch[:,:,3:0:-1] / 3000
    if sum(cloud_free_patches) > 0:
        aggregate_pred = np.ma.median(np.array(preds)[cloud_free_patches], axis=0)
        mean_patch[aggregate_pred > threshold, 0] = 0.9
        mean_patch[aggregate_pred > threshold, 1] = 0
        mean_patch[aggregate_pred > threshold, 2] = 0.1
    plt.subplot(num_img, num_img, i + 2)
    plt.title('Median', size=5, y=0.9)
    plt.imshow(np.clip(mean_patch, 0, 1))
    plt.axis('off')
    if title:
        plt.suptitle(title, size=num_img * 2, y=0.93)
    plt.tight_layout()
    if path:
        plt.savefig(path + '.png', bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [ ]:
RECT_WIDTH = 0.004
START_DATE = '2019-01-01'
END_DATE = '2021-06-01'
MOSAIC_PERIOD = 3
SPECTROGRAM_INTERVAL = 2

In [ ]:
ensemble_name = 'v0.0.11_ensemble-8-25-21'
model_list = dl_utils.load_ensemble(f'../models/{ensemble_name}')

model = keras.models.load_model('../models/spectrogram_v0.0.11_2021-07-13.h5')

patch_model = keras.models.load_model('../models/v2.0_weak_labels_28x28x24.h5', custom_objects={'LeakyReLU': keras.layers.LeakyReLU,
                                                'ELU': keras.layers.ELU,
                                                'ReLU': keras.layers.ReLU
                                               }) 

In [ ]:
coord = [108.060, -6.323]

In [ ]:
mosaics, metadata = dl_utils.download_mosaics(
                        dl_utils.rect_from_point(coord, RECT_WIDTH), 
                        START_DATE, 
                        END_DATE, 
                        MOSAIC_PERIOD, 
                        method='min')
dates = [m['metadata']['']['id'][15:25] for m in metadata]
pairs, pair_dates = dl_utils.pair(mosaics, SPECTROGRAM_INTERVAL, dates=dates)
#preds = [dl_utils.predict_spectrogram(pair, model) for pair in pairs] # if predicting single model
preds = dl_utils.predict_ensemble(pairs, model_list)
patch_stack = []
for pair in pairs:
    model_input = np.zeros((28,28,24))
    model_input[:,:,:12] = dl_utils.unit_norm(dl_utils.pad_patch(pair[0][9:,9:].filled(0), 28))
    model_input[:,:,12:] = dl_utils.unit_norm(dl_utils.pad_patch(pair[1][9:,9:].filled(0), 28))
    patch_stack.append(model_input)
patch_preds = patch_model.predict(np.array(patch_stack))[:,1]
subtitles = [f"{date[:7]} - {patch_pred:.2f}" for date, patch_pred in zip(pair_dates, patch_preds)]
patches = [np.mean(pair, axis=0) for pair in pairs]
title = f'{coord[0]:.3f}, {coord[1]:.3f}'
path = f'../figures/neural_network/{title}'
visualize_time_series(pairs, preds, subtitles, threshold=0.8, title=title, path=path)

## Visualize Classifier Outputs for a List of Sites

In [ ]:
import geopandas as gpd
site_file = gpd.read_file('/Users/ckruse/Downloads/detections.geojson')

In [ ]:
for site in tqdm(site_file['geometry']):
    coords = [site.x, site.y]
    mosaics, metadata = dl_utils.download_mosaics(
                        dl_utils.rect_from_point(coords, RECT_WIDTH), 
                        START_DATE, 
                        END_DATE, 
                        MOSAIC_PERIOD, 
                        method='min')
    dates = [m['metadata']['']['id'][15:25] for m in metadata]
    pairs, pair_dates = dl_utils.pair(mosaics, SPECTROGRAM_INTERVAL, dates=dates)
    preds = [dl_utils.predict_spectrogram(pair, model) for pair in pairs]
    patch_stack = []
    for pair in pairs:
        model_input = np.zeros((28,28,24))
        model_input[:,:,:12] = dl_utils.unit_norm(dl_utils.pad_patch(pair[0][9:,9:].filled(0), 28))
        model_input[:,:,12:] = dl_utils.unit_norm(dl_utils.pad_patch(pair[1][9:,9:].filled(0), 28))
        patch_stack.append(model_input)
    patch_preds = patch_model.predict(np.array(patch_stack))[:,1]
    subtitles = [f"{date[:7]} - {patch_pred:.2f}" for date, patch_pred in zip(pair_dates, patch_preds)]
    patches = [np.mean(pair, axis=0) for pair in pairs]
    title = f'{coords[0]:.3f}, {coords[1]:.3f}'
    path = f'../figures/found_sites/clf-2.0-{title}'
    visualize_time_series(pairs, preds, subtitles, threshold=0.6, title=title, path=path)